In [2]:
# 샘플 데이터 로딩
from sklearn.datasets import load_iris 

dataset = load_iris()

x = dataset['data']
y = dataset['target']

In [3]:
from sklearn.utils import resample
import numpy as np
from sklearn.model_selection import train_test_split


# data sampling
x_sampled, y_sampled = resample(x, y, n_samples=100, random_state=42)

print(x_sampled.shape)
print(y_sampled.shape)


# data split
x_training, x_test, y_training, y_test = train_test_split(x_sampled, y_sampled, test_size=0.2, shuffle=True, stratify=y_sampled, random_state=34)
x_train, x_valid, y_train, y_valid = train_test_split(x_training, y_training, test_size=0.2, shuffle=True, stratify=y_training, random_state=34)

(100, 4)
(100,)


In [ ]:
# 모델 코드 작성
import torch
import torch.nn as nn
import torch.nn.functional as F


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Skip connection adjustment
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.skip(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        out = F.relu(out)
        return out


class ResNetLikeCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNetLikeCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = ResidualBlock(16, 32, stride=2)
        self.layer2 = ResidualBlock(32, 64, stride=2)
        self.layer3 = ResidualBlock(64, 128, stride=2)
        self.gap = nn.AdaptiveAvgPool2d(1)  # Global Average Pooling
        self.fc = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.gap(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNetLikeCNN(num_classes=10)
model.to(device)

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomResizedCrop(32),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
# loss function
import matplotlib.pyplot as plt

# 손실 감소 그래프
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.grid()
plt.title('Loss over Epochs')
plt.xlabel('epoch')
plt.ylabel('Loss')

In [ ]:
## data evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
pred=model(y_test)


# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat)


# 정확도
accuracy=accuracy_score(y_true=y_test, y_pred=pred)
print('정확도: ', accuracy)


# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred)
print('정밀도: ', precision)


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred)
print('재현율: ', recall)


# roc auc
fpr, tpr, _ = roc_curve(y_true=y_test, y_pred=pred)

plt.plot(fpr, tpr)
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')

plt.show()

print('auc 점수: ', roc_auc_score(y_test, pred))

In [ ]:
# 적용 예시


